# <center> Homework 2 </center>

In [1]:
# autoreload позволяет нам подгружать изменения во внешнем коде
# без необходимости перезагружать kernel у ноутбука
%load_ext autoreload
%autoreload 2

## 1) Exceptions
В модуле `exceptions` объявите следующие исключения:
- LowFuelError
- NotEnoughFuel
- CargoOverload

In [2]:
import exceptions

## 2) Vehicle 

<img src="https://cs.pikabu.ru/images/big_size_comm/2013-12_4/13874526438897.jpg" width=300>

Доработайте базовый класс `Vehicle`:
- добавьте атрибуты `weight`, `started`, `fuel`, `fuel_consumption` со значениями по умолчанию
- добавьте инициализатор для установки `weight`, `fuel`, `fuel_consumption`
- добавьте метод `start`, который, если ещё не состояние `started`, проверяет, что топлива больше нуля, 
  и обновляет состояние `started`, иначе выкидывает исключение `exceptions.LowFuelError`
- добавьте метод `move`, который проверяет, 
  что достаточно топлива для преодоления переданной дистанции, 
  и изменяет количество оставшегося топлива, иначе выкидывает исключение `exceptions.NotEnoughFuel`

In [3]:
from abc import ABC


class Vehicle(ABC):
    """
    fuel_consumption: how much fuel we waste if we move distance=1
    """

    def __init__(self, weight=0, fuel=0, fuel_consumption=0):
        self.weight = weight
        self._started = False
        self.fuel = fuel
        self.fuel_consumption = fuel_consumption
    
    @property
    def started(self):
        return self._started

    def start(self):
        if self.fuel <= 0:
            raise exceptions.LowFuelError
        self._started = True

    def move(self, distance):
        # нет в задании, но логично проверить, 
        # что дистанция неотрицательная, иначе у нас добавляется топливо
        if distance < 0:
            raise ValueError("distance must be non-negative")
        fuel_to_burn = distance * self.fuel_consumption
        if fuel_to_burn > self.fuel:
            raise exceptions.NotEnoughFuel
        self.fuel -= fuel_to_burn


In [4]:
# тесты для проверки класса Vehicle
import pytest

v = Vehicle(fuel=90)
assert v.started is False
v.start()
assert v.started is True


v = Vehicle(fuel=0)
with pytest.raises(exceptions.LowFuelError):
    v.start()
assert v.started is False


v = Vehicle(fuel=90, fuel_consumption=10)
v.move(8)
assert v.fuel == pytest.approx(10, 1e-8)


v = Vehicle(fuel=70, fuel_consumption=10)
with pytest.raises(exceptions.NotEnoughFuel):
    v.move(8)
assert v.fuel == 70


v = Vehicle(fuel=70, fuel_consumption=10)
with pytest.raises(ValueError):
    v.move(-5)
assert v.fuel == 70

## 3) Engine 

<img src="https://upload.wikimedia.org/wikipedia/commons/6/6f/National_gas_engine_%28Rankin_Kennedy%2C_Modern_Engines%2C_Vol_II%29.jpg" width=250>

Создайте датакласс `Engine`, добавьте атрибуты `volume` и `pistons`

In [5]:
class Engine:
    def __init__(self, volume, pistons):
        self._volume = volume
        self._pistons = pistons
        
    @property
    def volume(self):
        return self._volume
    
    @property
    def pistons(self):
        return self._pistons

## 4) Car 

<img src="https://lh3.googleusercontent.com/proxy/RJqzSJqZFQrx7xOXimZQ4ooInGE6ViJdwz7JZZ_b__Eun-stRH2NMw47fM4je2hL1afRtmpKS9kZUVY0p2YcaZVtJ7bKRw3wWHRRa6zVUbeIb5TYLNkwx5A" width=250>

Cоздайте класс `Car`
    - класс `Car` должен быть наследником `Vehicle`
    - добавьте атрибут `engine` классу `Car`
    - объявите метод `set_engine`, который принимает в себя экземпляр объекта `Engine` и устанавливает на текущий экземпляр `Car`

In [6]:
class Car(Vehicle):
    @property
    def engine(self):
        return self._engine
    
    def set_engine(self, engine: Engine):
        self._engine = engine

In [7]:
# тесты для проверки класса Car
import pytest

car = Car()
engine = Engine(volume=3000, pistons=4)
car.set_engine(engine)
assert engine is car.engine

## 5) Plane

<img src="http://www.weirduniverse.net/images/2017/1934parachute02.jpg" width=250>

Создайте класс `Plane`
- класс `Plane` должен быть наследником `Vehicle`
- добавьте атрибуты `cargo` и `max_cargo` классу `Plane`
- добавьте `max_cargo` в инициализатор (переопределите родительский)
- объявите метод `load_cargo`, который принимает число, проверяет, что в сумме с текущим `cargo` не будет перегруза, и обновляет значение, в ином случае выкидывает исключение `exceptions.CargoOverload`
- объявите метод `remove_all_cargo`, который обнуляет значение `cargo` и возвращает значение `cargo`, которое было до обнуления

In [8]:
class Plane(Vehicle):
    def __init__(self, max_cargo, weight=0, fuel=0, fuel_consumption=0):
        Vehicle.__init__(self, weight=weight, fuel=fuel, fuel_consumption=fuel_consumption)
        self.max_cargo = max_cargo
        self._cargo = 0
        
    @property
    def cargo(self):
        return self._cargo

    def load_cargo(self, cargo):
        if (cargo + self._cargo > self.max_cargo):
            raise exceptions.CargoOverload
        self._cargo += cargo
        
    def remove_all_cargo(self):
        result = self._cargo
        self._cargo = 0
        return result

In [9]:
# тесты для проверки класса Plane
import pytest

plane = Plane(max_cargo=1000)
assert plane.cargo == 0
plane.load_cargo(300)
assert plane.cargo == 300
plane.load_cargo(699)
assert plane.cargo == 999
with pytest.raises(exceptions.CargoOverload):
    plane.load_cargo(2)

    
plane = Plane(max_cargo=1000)
plane.load_cargo(401)
plane.load_cargo(5)
result = plane.remove_all_cargo()
assert 406 == pytest.approx(result, 1e-8)